# Prepare input data for protected areas  

Notebook with the code to:
- check if provided data matches the required data model format (test)  
- format the data according to the expected data model
- edit/match countries names
- save csv file for the widget

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pandera as pa
from pandera.typing import Series
import requests
from pathlib import Path
import os
import logging

In [17]:
!pip install openpyxl

     |████████████████████████████████| 242 kB 10.7 MB/s eta 0:00:01


## Load and process Mangroove data

Modify mangrove data (area by country) to fit into the new data model, and then check the validation.

In [44]:
mang_df = pd.read_excel('../../../datasets/Mangrove_Protection_Calculations_20210430.xlsx', sheet_name='National')

mang_df.head()

,Country,Total Mangrove Composite,Total Protected Mangrove Composite,Total Mangrove 1996,Total Protected Mangrove 1996,Total Mangrove 2007,Total Protected Mangrove 2007,Total Mangrove 2010,Total Protected Mangrove 2010,Total Mangrove 2016,Total Protected Mangrove 2016,Net Change in Total Mangrove Extent,Net Change in Protected Mangrove Extent,Unnamed: 13,% in protected areas in 1996,% in protected areas in 2007,% in protected areas in 2010,% protected in 2016
0,American Samoa,0.187447,0.000000,0.187447,0.000000,0.187447,0.000000,0.187447,0.000000,0.187447,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
1,Angola,139.542529,1.769203,136.884844,1.725696,133.431203,1.715800,129.155410,1.611917,132.828293,1.687598,-4.056551,-0.038098,NaN,0.012607,0.012859,0.012705,0.012705
2,Anguilla,0.021393,0.000000,0.021393,0.000000,0.021393,0.000000,0.008674,0.000000,0.008674,0.000000,-0.012719,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
3,Antigua and Barbuda,9.064915,4.494175,9.048649,4.479555,9.048649,4.479555,8.856625,4.435916,8.863024,4.444657,-0.185625,-0.034898,NaN,0.495052,0.495052,0.501483,0.501483
4,Aruba,0.543387,0.008297,0.543387,0.008297,0.337894,0.008297,0.337894,0.008297,0.337894,0.008297,-0.205493,0.000000,NaN,0.015269,0.024555,0.024555,0.024555


In [46]:
mang_df = mang_df.filter(regex='Country|Total|Net|%').dropna()
mang_df.head()

,Country,Total Mangrove Composite,Total Protected Mangrove Composite,Total Mangrove 1996,Total Protected Mangrove 1996,Total Mangrove 2007,Total Protected Mangrove 2007,Total Mangrove 2010,Total Protected Mangrove 2010,Total Mangrove 2016,Total Protected Mangrove 2016,Net Change in Total Mangrove Extent,Net Change in Protected Mangrove Extent,% in protected areas in 1996,% in protected areas in 2007,% in protected areas in 2010,% protected in 2016
0,American Samoa,0.187447,0.000000,0.187447,0.000000,0.187447,0.000000,0.187447,0.000000,0.187447,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Angola,139.542529,1.769203,136.884844,1.725696,133.431203,1.715800,129.155410,1.611917,132.828293,1.687598,-4.056551,-0.038098,0.012607,0.012859,0.012705,0.012705
2,Anguilla,0.021393,0.000000,0.021393,0.000000,0.021393,0.000000,0.008674,0.000000,0.008674,0.000000,-0.012719,0.000000,0.000000,0.000000,0.000000,0.000000
3,Antigua and Barbuda,9.064915,4.494175,9.048649,4.479555,9.048649,4.479555,8.856625,4.435916,8.863024,4.444657,-0.185625,-0.034898,0.495052,0.495052,0.501483,0.501483
4,Aruba,0.543387,0.008297,0.543387,0.008297,0.337894,0.008297,0.337894,0.008297,0.337894,0.008297,-0.205493,0.000000,0.015269,0.024555,0.024555,0.024555


In [47]:
years = mang_df.filter(regex='[0-9]').columns.str[-4:]
years = list(set(years))
years

['1996', '2010', '2016', '2007']

In [50]:
for year in years:
    pat = 'Country|'+year
    year_df = mang_df.filter(regex=pat).copy()
    year_df.columns = ['Country', 'total_area', 'protected_area', 'year']
    year_df['year'] = np.int64(year)
    if year == years[0]:
        df_final = year_df
    else:
        df_final = pd.concat([df_final, year_df])
df_final

,Country,total_area,protected_area,year
0,American Samoa,0.187447,0.000000,1996
1,Angola,136.884844,1.725696,1996
2,Anguilla,0.021393,0.000000,1996
3,Antigua and Barbuda,9.048649,4.479555,1996
4,Aruba,0.543387,0.008297,1996
...,...,...,...,...
102,Vanuatu,17.700817,0.000000,2007
103,Venezuela,2803.137176,1888.500410,2007
104,Vietnam,1622.112847,732.712277,2007
105,"Virgin Islands, U.S.",2.067021,1.208262,2007


In [51]:
df_final.dtypes

Country            object
total_area        float64
protected_area    float64
year                int64
dtype: object

Check validation (without countries / location for now)

In [52]:
data_validation(df_final.filter(regex='area|year'))

variable total_area is OK
variable protected_area is OK
variable year is OK
ALL VARIABLES OK


## Check countries and location ids match

Load API's locations data 

In [75]:
#dataLocation = requests.get('https://mangrove-atlas-api.herokuapp.com/api/v2/locations').json()['data']
dataLocation = requests.get('http://192.168.50.115:3000/api/v2/locations').json()['data']
locations = pd.DataFrame(dataLocation)
locations

,id,iso,bounds,location_type,name,area_m2,perimeter_m,coast_length_m,location_id
0,1155,WORLDWIDE,None,worldwide,Worldwide,1.489400e+14,NaN,1.634701e+09,worldwide
1,1012,ARE,"{'coordinates': [[[55.46933090962499, 25.41780...",wdpa,Al Zorah,1.959136e+06,7.080970e+03,5.679490e+03,2_00000000000000000b7a
2,1114,BRA,"{'coordinates': [[[-52.414494393511234, -3.106...",wdpa,Amazon Estuary and its Mangroves,3.821925e+10,1.257113e+07,5.873673e+06,2_000000000000000009c2
3,992,AGO,"{'coordinates': [[[8.20187877548665, -18.01639...",country,Angola,1.744005e+12,7.368212e+06,2.007891e+06,1_2_97
4,964,ATG,"{'coordinates': [[[-62.753156503651574, 16.613...",country,Antigua & Barbuda,1.087145e+11,1.465019e+06,3.107418e+05,1_2_69
...,...,...,...,...,...,...,...,...,...
258,1140,CHN,"{'coordinates': [[[109.78242132123188, 21.4545...",wdpa,Zhanjiang Mangrove National Nature Reserve,2.261498e+07,2.160093e+04,9.758900e+03,2_00000000000000000baa
259,1067,ECU,"{'coordinates': [[[-81.09091088361897, -1.6829...",wdpa,Zona Marina Parque Nacional Machalilla,5.394775e+08,1.176965e+05,4.530973e+04,2_0000000000000000028b
260,1061,MEX,"{'coordinates': [[[-92.50817154426318, 14.7223...",wdpa,Zona Sujeta a Conservación Ecológica Cabildo -...,2.842541e+07,3.042521e+04,7.380090e+03,2_000000000000000007b9
261,1062,MEX,"{'coordinates': [[[-92.3919986127088, 14.53338...",wdpa,Zona Sujeta a Conservación Ecológica El Gancho...,4.688550e+07,7.230276e+04,1.225422e+04,2_000000000000000007ba


In [76]:
country_filter = df_final.Country.unique()
print(f'Total of {len(country_filter)} countries')

Total of 107 countries


In [77]:
print('Countries with match on locations file:')
df_final[(df_final['Country'].isin(locations['name'])) & (df_final['year'] == 2007)]


Countries with match on locations file:


,Country,total_area,protected_area,year
1,Angola,133.431203,1.715800,2007
5,Australia,9857.920227,4946.605567,2007
7,Bahrain,0.651996,0.000000,2007
8,Bangladesh,4137.246296,3790.279334,2007
10,Belize,464.941544,141.812582,2007
...,...,...,...,...
101,United States,1981.861799,1769.233058,2007
102,Vanuatu,17.700817,0.000000,2007
103,Venezuela,2803.137176,1888.500410,2007
104,Vietnam,1622.112847,732.712277,2007


In [78]:
dif = df_final[-(df_final['Country'].isin(locations['name'])) & (df_final['year'] == 2007)]
print('Countries with NO match on locations file')
print(f'{len(dif)} contries:')
dif



Countries with NO match on locations file
23 contries:


,Country,total_area,protected_area,year
0,American Samoa,0.187447,0.000000,2007
2,Anguilla,0.021393,0.000000,2007
3,Antigua and Barbuda,9.048649,4.479555,2007
4,Aruba,0.337894,0.008297,2007
6,Bahamas,1030.588846,656.930166,2007
9,Barbados,0.136871,0.000000,2007
12,"Bonaire, Saint Eustatius and Saba",1.819701,1.819698,2007
14,British Virgin Islands,0.887694,0.000000,2007
18,Cayman Islands,42.106559,9.310517,2007
23,Côte d'Ivoire,62.168093,4.582776,2007


First, let's explore the cases where the difference is in the words **and**, **Island**, **Saint** or **The**

In [79]:
locations[(locations['name'].str.contains('&|Is.|The|the|St.')) & (locations['location_type'] == 'country')]

,id,iso,bounds,location_type,name,area_m2,perimeter_m,coast_length_m,location_id
4,964,ATG,"{'coordinates': [[[-62.753156503651574, 16.613...",country,Antigua & Barbuda,1.087145e+11,1.465019e+06,3.107418e+05,1_2_69
26,900,CYM,"{'coordinates': [[[-83.5972199608967, 17.58408...",country,Cayman Is.,1.197471e+11,1.327342e+06,3.397607e+05,1_2_5
203,955,VCT,"{'coordinates': [[[-63.378358224530146, 12.036...",country,Saint Vincent and the Grenadines,3.684079e+10,8.686558e+05,2.836896e+05,1_2_60
225,984,SLB,"{'coordinates': [[[154.58555580173928, -16.126...",country,Solomon Is.,1.633613e+12,5.852608e+06,1.059956e+07,1_2_89
229,934,KNA,"{'coordinates': [[[-63.629026517390606, 16.348...",country,St. Kitts & Nevis,1.051079e+10,4.414889e+05,1.321121e+05,1_2_39
230,936,LCA,"{'coordinates': [[[-62.81388945776756, 13.2366...",country,St. Lucia,1.614637e+10,6.924528e+05,2.121044e+05,1_2_41
239,969,BHS,"{'coordinates': [[[-81.2152796896022, 20.36826...",country,The Bahamas,6.097739e+11,3.680410e+06,1.595217e+07,1_2_74
240,921,GMB,"{'coordinates': [[[-20.181980575461676, 13.055...",country,The Gambia,3.353300e+10,1.548981e+06,1.956420e+05,1_2_26
243,987,TTO,"{'coordinates': [[[-62.08305618816621, 9.83194...",country,Trinidad & Tobago,8.082952e+10,1.312662e+06,7.784338e+05,1_2_92
245,912,TCA,"{'coordinates': [[[-72.81360640820247, 20.5452...",country,Turks & Caicos Is.,1.549139e+11,1.554640e+06,7.805141e+05,1_2_17


Try again after substitutions

In [81]:
replacement = {' and ':' & ', 'The ':'', 'Islands': 'Is.', 'Island': 'Is.'}
df_final_test= df_final.replace({'Country': replacement}, regex=True)

dif = df_final_test[-(df_final_test['Country'].isin(locations['name'])) & (df_final_test['year'] == 2007)]
print('Countries with NO match on locations file after initial substitutions')
print(f'{len(dif)} contries:')
dif


Countries with NO match on locations file after initial substitutions
19 contries:


,Country,total_area,protected_area,year
0,American Samoa,0.187447,0.000000,2007
2,Anguilla,0.021393,0.000000,2007
4,Aruba,0.337894,0.008297,2007
6,Bahamas,1030.588846,656.930166,2007
9,Barbados,0.136871,0.000000,2007
12,"Bonaire, Saint Eustatius & Saba",1.819701,1.819698,2007
14,British Virgin Is.,0.887694,0.000000,2007
23,Côte d'Ivoire,62.168093,4.582776,2007
25,Curaçao,0.142139,0.051602,2007
26,Democratic Republic of the Congo,497.438655,251.139272,2007


Extend replacement using (hard-coded) dictionary to replace countries names when available  

In [82]:
replacement = {'Bahamas': 'The Bahamas',
              "Côte d'Ivoire": "Cote d'Ivoire",
              'Bonaire, Saint Eustatius & Saba': 'Bonaire, Sint-Eustasius, Saba',
              'British Virgin Is.': 'Virgin Islands, British',
              'Democratic Republic of the Congo': 'Congo, DRC',
              'East Timor': 'Timor-Leste',
               'Gambia':'The Gambia',
              'Saint Kitts & Nevis': 'St. Kitts & Nevis',
              'Saint Lucia': 'St. Lucia',
              'Saint Vincent & the Grenadines':'Saint Vincent and the Grenadines',
              'Virgin Is., U.S.':'United States Virgin Islands'}

In [83]:
df_final_fixed= df_final.replace({'Country': {' and ':' & ', 'The ':'', 'Islands': 'Is.', 'Island': 'Is.'}}, regex=True)
df_final_fixed= df_final_fixed.replace({'Country': replacement}, regex=True)

dif = df_final_fixed[-(df_final_fixed['Country'].isin(locations['name'])) & (df_final_fixed['year'] == 2007)]
print('Countries with NO match on locations file after substitutions')
print(f'{len(dif)} contries:')
dif

Countries with NO match on locations file after substitutions
8 contries:


,Country,total_area,protected_area,year
0,American Samoa,0.187447,0.000000,2007
2,Anguilla,0.021393,0.000000,2007
4,Aruba,0.337894,0.008297,2007
9,Barbados,0.136871,0.000000,2007
25,Curaçao,0.142139,0.051602,2007
28,Dominica,0.017582,0.017582,2007
49,Hong Kong,4.464429,1.552409,2007
83,Sao Tome & Principe,0.004863,0.000000,2007


In [84]:
rem = locations[-(locations['name'].isin(df_final_fixed['Country'])) & (locations['location_type'] == 'country')]
print('Countries in the location file with no data from the countries file')
print(f'{len(rem)} countries:')
rem

Countries in the location file with no data from the countries file
4 countries:


,id,iso,bounds,location_type,name,area_m2,perimeter_m,coast_length_m,location_id
170,980,PER,"{'coordinates': [[[-84.69587559769572, -20.140...",country,Peru,2.130703e+12,8.368277e+06,3431501.41,1_2_85
178,962,-,"{'coordinates': [[[142.05945935487523, -9.7551...",country,Protected zone Australia/Papua New Guinea,3.946704e+09,2.755621e+05,169704.81,1_2_67
201,938,MAF,"{'coordinates': [[[-63.65798392437718, 17.6407...",country,Saint Martin,5.455433e+09,4.388372e+05,133072.84,1_2_43
218,983,SGP,"{'coordinates': [[[103.59604294305795, 1.13035...",country,Singapore,1.363104e+09,1.621760e+05,506487.60,1_2_88


## Generate final table

In [86]:
df_save = df_final_fixed.merge(locations, how ='inner',left_on='Country', right_on='name')
df_save = df_save.filter(['location_id', 'total_area', 'protected_area', 'year'])
# df_save['metadata'] = str({'units':'ha'})

df_save

,location_id,total_area,protected_area,year
0,1_2_97,136.884844,1.725696,1996
1,1_2_97,129.155410,1.611917,2010
2,1_2_97,132.828293,1.687598,2016
3,1_2_97,133.431203,1.715800,2007
4,1_2_69,9.048649,4.479555,1996
...,...,...,...,...
391,1_2_96,2.067021,1.208262,2007
392,1_2_65,15.265364,2.353974,1996
393,1_2_65,15.410758,2.417552,2010
394,1_2_65,15.419600,2.417552,2016


In [88]:
df_save.to_csv('../../../datasets/processed/Mangrove_protected_area.csv', sep=',')

## v2 - use the new data model from a new source

In [7]:
logging.basicConfig(level=logging.INFO)
#  FIXME: This will depends from where the notebook kernel is running so be careful
# TODO: everything is currently against staging
PROD_API = 'https://mangrove-atlas-api.herokuapp.com'
STAGING_API = 'http://mangrove-atlas-api-staging.herokuapp.com'
WORK_DIR =  Path(os.getcwd())
BASE_DIR = Path(f'{WORK_DIR.parents[4]}/work/datasets')

# input_file = Path(f'{BASE_DIR}/widget_wdpa_data/mangroves_country_output_2022-06-01.csv')
input_file = Path(f'{BASE_DIR}/widget_wdpa_data/Mangrove in WDPA July 2022.xlsx')
output_file = Path(f'{BASE_DIR}/processed/Mangrove_protected_area_v2.csv')
# @TODO: Add expected data files source as an environment variable.
expected_dir = Path('/home/jovyan/work/datasets')
assert str(BASE_DIR) == str(expected_dir), f'{BASE_DIR} is not {expected_dir}'

In [15]:
class OutputSchema(pa.SchemaModel):
    year: Series[int] = pa.Field(nullable=False, ge=1996, le=2016)
    total_area: Series[float] = pa.Field(nullable=True,ge=0., le=10000000000.)
    protected_area: Series[float] = pa.Field(nullable=True, ge=0., le=10000000000.)
    location_id: Series[str] = pa.Field(nullable=False, allow_duplicates=True)
    # checks while trying to generate a strategy and a example seems to fail, i need to investigate further
    @pa.check("location_id")
    def location_id_check(cls, series: Series[str]) -> Series[bool]:
        """Check that location_id is a valid location_id"""
        dataLocation = requests.get(f'{PROD_API}/api/v2/locations').json()['data']
        loc = pd.DataFrame(dataLocation)[['id', 'name','iso', 
        'location_id', 'location_type']].query("location_type=='country'")
        return series.isin(loc.location_id.unique())

def read_data(file: Path) -> pd.DataFrame:
    """Read data from a file and return a pandas DataFrame"""
    if file.suffix == '.csv':
        client_provided = pd.read_csv(file)
    elif file.suffix == '.xlsx':
        client_provided = pd.read_excel(file)
    else:
        raise ValueError(f'{filePath} is not a valid file type')
    
    return client_provided

def validationPipe(filePath: Path, schema: OutputSchema = OutputSchema) -> None:
    """
    Transform the dataframe from the excel file to a pandas dataframe
    Args:
        filePath (str): path to the excel file
        sheet (str, optional): name of the sheet in the excel file. Defaults to 'Sheet3'.

    Returns:
        str: path to the transformed dataframe
    """
    client_provided = read_data(filePath)
    
    # do any transformation here

    validated = schema.validate(client_provided)
    
    if not validated:
        raise ValueError('The data is not valid')

def uploadDataApi(filePath: str, endpoint: str) -> str:
    """
    upload the data to the api
    Args:
        filePath (str): path to the file to upload
        endpoint (str): endpoint to upload to

    Returns:
        str: url of the uploaded file
    """
    files = {'file': open(filePath, 'rb')}
    r = requests.post(endpoint, files=files)
    return r.status_code

In [18]:
 validationPipe(input_file)

SchemaError: column 'year' not in dataframe
  ISO_TER  TOTAL MANGROVE AREA(HA)  MANGROVE AREA PROTECTED(HA)  \
0     ABW                45.938704                          1.0   
1     AGO             28356.673109                        146.0   
2     AIA                 3.700790                          0.0   
3     ARE              7444.860192                       1323.0   
4     ASM                32.050071                          0.0   

   PERCENT PROTECTED  
0           0.021768  
1           0.005149  
2           0.000000  
3           0.177706  
4           0.000000  

In [19]:
### Read location ids file
dataLocation = requests.get(f'{PROD_API}/api/v2/locations').json()['data']
loc = pd.DataFrame(dataLocation)[['id', 'name','iso', 
'location_id', 'location_type']].query("location_type=='country'")
loc.head(2)

,id,name,iso,location_id,location_type
3,992,Angola,AGO,1_2_97,country
4,964,Antigua & Barbuda,ATG,1_2_69,country


In [29]:
protected_data = read_data(input_file)
protected_data.rename(columns = str.lower, inplace = True)
protected_data['year'] = 2016
protected_data.head()

,iso_ter,total mangrove area(ha),mangrove area protected(ha),percent protected,year
0,ABW,45.938704,1.0,0.021768,2016
1,AGO,28356.673109,146.0,0.005149,2016
2,AIA,3.700790,0.0,0.000000,2016
3,ARE,7444.860192,1323.0,0.177706,2016
4,ASM,32.050071,0.0,0.000000,2016


In [30]:
merged = pd.merge(protected_data, loc,how='inner', left_on='iso_ter', right_on='iso')
merged.rename(columns = {'total_area_2016':'total_area', 
                         'total mangrove area(ha)':'total_area',
                         'mangrove area protected(ha)':'protected_area',
                        }, inplace = True)
filter_merged = merged[['total_area', 'protected_area', 'location_id','year']]
filter_merged.head(2)

,total_area,protected_area,location_id,year
0,28356.673109,146.0,1_2_97,2016
1,7444.860192,1323.0,1_2_68,2016


In [31]:
OutputSchema.validate(filter_merged)
filter_merged.to_csv(output_file, index=False)

In [32]:
merged.query('iso=="TZA"')

,iso_ter,total_area,protected_area,percent protected,year,id,name,iso,location_id,location_type
90,TZA,110787.165749,90598.0,0.817766,2016,917,Tanzania,TZA,1_2_22,country


In [33]:
protected_data.iso_ter.count()

123

In [34]:
merged.iso_ter.count()

101

In [35]:
#TODO: we have missing locations, this means that we need to add them in the location table

In [36]:
#','.join(protected_data.query("protected_area>0").iso_ter.unique())

In [37]:
uploadDataApi(output_file, f'{PROD_API}/api/v2/widgets/protected-areas/import')

201